### Get data

Gets the Fashion MNIST data from the cloud and saves it under the "data"
directory.

In [1]:
from torchvision import datasets
from torchvision.transforms import ToTensor

DATA_DIR = "../data"

datasets.FashionMNIST(
    root=DATA_DIR,
    train=True,
    download=True,
    transform=ToTensor(),
)

datasets.FashionMNIST(
    root=DATA_DIR,
    train=False,
    download=True,
    transform=ToTensor(),
)


/home/bstollnitz/miniconda3/envs/aml-command-output/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


26422272it [00:02, 10175754.30it/s]                              


Extracting ../data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



29696it [00:00, 186288.88it/s]                          


Extracting ../data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



...

Extracting ../data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



6144it [00:00, 8673781.14it/s]          

Extracting ../data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw

Processing...
Done!



/home/bstollnitz/miniconda3/envs/aml-command-output/lib/python3.9/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1616554782469/work/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: ../data
    Split: Test
    StandardTransform
Transform: ToTensor()

### Train

Splits the training data into training and validation sets, uses it to train
the model, and saves it under the "model" directory.

In [2]:
from pathlib import Path
from typing import Tuple

import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor

from neural_network import NeuralNetwork
from utils_train_nn import fit, evaluate

MODEL_DIR = "../model_from_notebook"

def load_train_val_data(data_dir: str, batch_size: int, training_fraction: float
    ) -> Tuple[DataLoader, DataLoader]:
    """
    Returns two DataLoader objects that wrap training and validation data.
    Training and validation data are extracted from the full original training
    data, split according to training_fraction.
    """
    full_train_data = datasets.FashionMNIST(data_dir,
                                            train=True,
                                            download=False,
                                            transform=ToTensor())
    full_train_len = len(full_train_data)
    train_len = int(full_train_len * training_fraction)
    val_len = full_train_len - train_len
    (train_data, val_data) = random_split(dataset=full_train_data,
                                          lengths=[train_len, val_len])
    train_loader = DataLoader(train_data,
                              batch_size=batch_size,
                              shuffle=True)
    val_loader = DataLoader(val_data,
                            batch_size=batch_size,
                            shuffle=True)

    return (train_loader, val_loader)


def train(data_dir: str, model_dir: str, device: str) -> None:
    """
    Trains the model for a number of epochs, and saves it.
    """
    learning_rate = 0.1
    batch_size = 64
    epochs = 5

    (train_dataloader, val_dataloader) = load_train_val_data(data_dir,
        batch_size, 0.8)

    model = NeuralNetwork()
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    print("\n***Training***")
    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}")

        (training_loss, training_accuracy) = fit(device, train_dataloader,
                                                 model, loss_fn, optimizer)
        print(f"Train loss: {training_loss:8f}, train accuracy:" +
            f"{training_accuracy * 100:0.1f}%")
        
        (validation_loss, validation_accuracy) = evaluate(device,
            val_dataloader, model, loss_fn)
        print(f"Validation loss: {validation_loss:8f}, validation accuracy:" +
            f"{validation_accuracy * 100:0.1f}%")

        print("\n-------------------------------")

    save_model(model_dir, model)


def save_model(model_dir, model: nn.Module) -> None:
    """
    Saves the trained model.
    """
    Path(model_dir).mkdir(parents=True, exist_ok=True)
    path = Path(model_dir, "weights.pth")
    print(f"\nSaving model to {path}")
    torch.save(model.state_dict(), path)


device = "cuda" if torch.cuda.is_available() else "cpu"
train(DATA_DIR, MODEL_DIR, device)


***Training***

Epoch 1


100%|██████████| 750/750 [00:05<00:00, 145.05it/s]


Train loss: 0.011057, train accuracy:75.0%
Validation loss: 0.008505, validation accuracy:79.3%

-------------------------------

Epoch 2


100%|██████████| 750/750 [00:05<00:00, 128.53it/s]


Train loss: 0.007550, train accuracy:82.7%
Validation loss: 0.006836, validation accuracy:84.6%

-------------------------------

Epoch 3


...

Train loss: 0.006924, train accuracy:84.1%
Validation loss: 0.006910, validation accuracy:84.1%

-------------------------------

Epoch 4


100%|██████████| 750/750 [00:04<00:00, 150.83it/s]


Train loss: 0.006549, train accuracy:85.1%
Validation loss: 0.006298, validation accuracy:85.4%

-------------------------------

Epoch 5


100%|██████████| 750/750 [00:05<00:00, 130.70it/s]


Train loss: 0.006294, train accuracy:85.5%
Validation loss: 0.006840, validation accuracy:84.0%

-------------------------------

Saving model to ../model_from_notebook/weights.pth


### Test with test data

Checks the loss and accuracy using our test data. 

In [3]:
batch_size = 64

test_data = datasets.FashionMNIST(DATA_DIR,
                                  train=False,
                                  download=False,
                                  transform=ToTensor())
test_loader = DataLoader(test_data,
                         batch_size=batch_size,
                         shuffle=True)

model = NeuralNetwork()
model_path = Path(MODEL_DIR, "weights.pth")
model.load_state_dict(torch.load(model_path))

loss_fn = nn.CrossEntropyLoss()

print('\n***Evaluating***')
(test_loss, test_accuracy) = evaluate(device, test_loader, model, loss_fn)
print(f'Test loss: {test_loss:>8f}, ' +
    f'test accuracy: {test_accuracy * 100:>0.1f}%')


***Evaluating***
Test loss: 0.007409, test accuracy: 82.4%


### Make a prediction with a single image

Opens the image in the "test_image" folder and predicts its label.

In [4]:
from PIL import Image
import numpy as np

from utils_score import predict

TEST_IMAGE_DIR = "../test_image"
LABELS_MAP = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

test_image_path = Path(TEST_IMAGE_DIR, "predict_image.png")

with Image.open(test_image_path) as image:
    x = np.asarray(image).reshape((-1, 28, 28)) / 255.0

predicted_index = predict(model, x, device)[0]
predicted_class = LABELS_MAP[predicted_index]
print(predicted_class)

Ankle Boot


### Generate sample data in csv for local mlflow prediction

In [5]:
delimiter = ","
fmt = "%.6f"
with Image.open(test_image_path) as image:
    x = np.asarray(image).reshape((1, -1)) / 255.0
    header = delimiter.join([f"col_{i}" for i in range(x.shape[1])])
    np.savetxt(fname=Path(TEST_IMAGE_DIR, "predict_image.csv"),
        X=x, delimiter=delimiter, fmt=fmt, header=header)


### Generate sample data in json for local mlflow prediction

In [6]:
from pandas import DataFrame

with Image.open(test_image_path) as image:
    x = np.asarray(image).reshape((1, -1)) / 255.0

    column_names = [f"col_{i}" for i in range(x.shape[1])]
    df = DataFrame(data=x, columns=column_names)

    data_json = df.to_json(orient="split")
    with open(Path(TEST_IMAGE_DIR, "predict_image.json"), "wt", encoding="utf-8") as file:
        file.write(data_json)

### Generate sample data in json for Azure ML prediction

In [7]:
import json

with Image.open(test_image_path) as image:
    x = np.asarray(image).reshape((1, -1)) / 255.0

    column_names = [f"col_{i}" for i in range(x.shape[1])]
    df = DataFrame(data=x, columns=column_names)

    data_json = '{"input_data":' + df.to_json(orient="split") + '}'
    with open(Path(TEST_IMAGE_DIR, "predict_image_azureml.json"), "wt", encoding="utf-8") as file:
        file.write(data_json)